# Import libraries

In [1]:
# drive access
from google.colab import drive
drive.mount('/content/drive')

# standard library
import numpy as np
import pandas as pd

# for audio
import librosa
from IPython.display import Audio

# for file streaming
import os

# for model
!pip install transformers
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torch

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.9 MB/s eta 0:00:00


In [2]:
# load model and tokenizer
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Load examples from bdl

In [7]:
audio, _ = librosa.load('/content/drive/MyDrive/266/Data/Raw_Data/Arctic/cmu_us_awb_arctic/wav/arctic_b0539.wav', sr=16000)

In [8]:
Audio(audio, rate=16000)

In [5]:
audio, _ = librosa.load('/content/drive/MyDrive/266/Data/Raw_Data/Arctic/cmu_us_awb_arctic/wav/arctic_b0538.wav', sr=16000)

In [6]:
Audio(audio, rate=16000)

# get the files from bea_Amused folder & create auto transcription using wav2vec

In [9]:
audio_filename = os.listdir('/content/drive/MyDrive/266/Data/Raw_Data/Arctic/cmu_us_awb_arctic/wav')

audio_filename

['arctic_a0126.wav',
 'arctic_b0029.wav',
 'arctic_a0221.wav',
 'arctic_a0104.wav',
 'arctic_b0020.wav',
 'arctic_a0066.wav',
 'arctic_a0115.wav',
 'arctic_a0163.wav',
 'arctic_a0111.wav',
 'arctic_a0146.wav',
 'arctic_a0178.wav',
 'arctic_a0075.wav',
 'arctic_b0012.wav',
 'arctic_a0002.wav',
 'arctic_a0125.wav',
 'arctic_a0197.wav',
 'arctic_a0010.wav',
 'arctic_a0069.wav',
 'arctic_a0160.wav',
 'arctic_a0186.wav',
 'arctic_a0231.wav',
 'arctic_a0105.wav',
 'arctic_b0036.wav',
 'arctic_a0067.wav',
 'arctic_b0028.wav',
 'arctic_a0229.wav',
 'arctic_a0131.wav',
 'arctic_b0022.wav',
 'arctic_a0210.wav',
 'arctic_a0096.wav',
 'arctic_a0099.wav',
 'arctic_b0008.wav',
 'arctic_a0166.wav',
 'arctic_a0232.wav',
 'arctic_a0181.wav',
 'arctic_a0100.wav',
 'arctic_a0223.wav',
 'arctic_b0003.wav',
 'arctic_a0165.wav',
 'arctic_a0124.wav',
 'arctic_a0027.wav',
 'arctic_a0169.wav',
 'arctic_a0078.wav',
 'arctic_a0151.wav',
 'arctic_b0018.wav',
 'arctic_a0007.wav',
 'arctic_a0201.wav',
 'arctic_a022

In [10]:
audio_df = pd.DataFrame()

audio_df['filename'] = audio_filename

audio_df.head()

,filename
0,arctic_a0126.wav
1,arctic_b0029.wav
2,arctic_a0221.wav
3,arctic_a0104.wav
4,arctic_b0020.wav


In [11]:
auto_transcription = []

for filename in audio_df['filename']:
  audio, _ = librosa.load(f'/content/drive/MyDrive/266/Data/Raw_Data/Arctic/cmu_us_awb_arctic/wav/{filename}', sr=16000)
  input_value = processor(audio, return_tensors="pt", padding="longest", sampling_rate=16000).input_values
  predicted_id = torch.argmax(model(input_value).logits, dim=-1)
  transcription = processor.batch_decode(predicted_id)
  auto_transcription.append(transcription[0])
  del audio, input_value, predicted_id, transcription

auto_transcription

['IT WAS JEAN SINGING SOFTLY OVER BEYOND THE ROCKS',
 'ALL OPERATIONS HAVE BEEN CARRIED ON FROM MONTRIOL ANTORONTO',
 'AT FIRST HE PUZZLED OVER SOMETHING UNTOWARD HE WAS SURE HAD HAPPENED',
 'HE WILL FOLLOW US SOON',
 'HE MADE NO REPLY AS HE WAITED FOR WHITMORE TO CONTINUE',
 'HE SAW JEANN FALTER FOR A MOMENT',
 'I SUPPOSED YOU PICKED THAT LINGL UP AMONG THE INDIANS',
 "TAKE AWAY THEIR FOREMEN AND THEY WOUDN'T BE WORT THEIR GRUB",
 'DO YOU KNOW THAT YOU ARE SHAKING MY CONFIDENCE IN YOU',
 'BUT THERE WAS SOMETHING EVEN MORE STARTLING THAN THIS RESEMBLANCE',
 'FOR THE FIRST TIME IN HIS LIFE HE WAS YARNING FOR A SCRAP',
 'AND NOW DOWN THERE ILEEN WAS WAITING FOR HIM',
 'SOMETHING AT WHITMORE HAD NOT YET SAID THRILLED HIM',
 'NOT AT THIS PARTICULAR CASE TOM APOLOGIZED WHITMORE',
 'SHE HAD DIED FROM COLD AND STARVATION',
 'BUT A STRANGE THING HAPPENED',
 "I'M PLAYING A SINGLE HAND IN WHAT LOOKS LIKE A LOSING GAME",
 'IT WAS HIS INTENTION TO RETURN TO ILIN AND HER FATHER',
 'DOES THAT LOOK G

confirm all files from bea_amused have been auto transcribed & add to the df

In [12]:
assert len(audio_df) == len(auto_transcription)

In [13]:
audio_df['auto_transcription'] = auto_transcription

In [14]:
audio_df.head()

,filename,auto_transcription
0,arctic_a0126.wav,IT WAS JEAN SINGING SOFTLY OVER BEYOND THE ROCKS
1,arctic_b0029.wav,ALL OPERATIONS HAVE BEEN CARRIED ON FROM MONTR...
2,arctic_a0221.wav,AT FIRST HE PUZZLED OVER SOMETHING UNTOWARD HE...
3,arctic_a0104.wav,HE WILL FOLLOW US SOON
4,arctic_b0020.wav,HE MADE NO REPLY AS HE WAITED FOR WHITMORE TO ...


take a look at one example

In [15]:
example = audio_df['filename'].iloc[0]
audio, _ = librosa.load(f'/content/drive/MyDrive/266/Data/Raw_Data/Arctic/cmu_us_awb_arctic/wav/{example}', sr=16000)

In [16]:
Audio(audio, rate=16000)

In [17]:
audio_df['auto_transcription'].iloc[0]

'IT WAS JEAN SINGING SOFTLY OVER BEYOND THE ROCKS'

save the df to csv file for future use

In [19]:
audio_df.to_csv('/content/drive/MyDrive/266/Data/Clean_Data/Arctic/auto_transcription/awb.csv', index=False)

In [20]:
check = pd.read_csv('/content/drive/MyDrive/266/Data/Clean_Data/Arctic/auto_transcription/awb.csv')

check.head()

,filename,auto_transcription
0,arctic_a0126.wav,IT WAS JEAN SINGING SOFTLY OVER BEYOND THE ROCKS
1,arctic_b0029.wav,ALL OPERATIONS HAVE BEEN CARRIED ON FROM MONTR...
2,arctic_a0221.wav,AT FIRST HE PUZZLED OVER SOMETHING UNTOWARD HE...
3,arctic_a0104.wav,HE WILL FOLLOW US SOON
4,arctic_b0020.wav,HE MADE NO REPLY AS HE WAITED FOR WHITMORE TO ...
